In [5]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
from datetime import datetime
url = 'https://www.meesho.com/accessories-men/pl/3tp'

response = r.get(url)

#saving in the file
html = response.text
with open("index.html",'w') as f:
    f.write(html)

In [9]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrap_meescho(url):
    # Make sure the URL is valid and not empty
    if not url:
        raise ValueError("URL cannot be empty")
    
    # Fetch the webpage
    response = r.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch URL. Status code: {response.status_code}")
    
    html_data = response.content
    soup = BeautifulSoup(html_data, 'lxml')
    
    # Define possible classes for product grids
    all_class = [
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA eCJiSA',
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA jGAjKy',
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA bGpzcx',
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA ihwYlH'
    ]
    
    # Initialize empty lists for scraped data
    Rating = []
    Products = []
    Price = []
    Reviews = []
    
    # Scrape data for each class
    for cls in all_class:
        for item in soup.find_all('div', {'class': cls}):
            try:
                prod, other_details = item.text.strip().split('₹', 1)
                rest_d, rat_rev = other_details.split('Delivery')
                pric = ''.join(filter(str.isdigit, rest_d.split()[0]))  # Extract price as digits only
                
                Products.append(prod.strip())
                Price.append(pric.strip())
                
                if 'supplier' in rat_rev.lower():
                    Rating.append(rat_rev[:3].strip())
                    Reviews.append(0)
                elif 'reviews' in rat_rev.lower():
                    rating_data = rat_rev.split()
                    Rating.append(rating_data[0].strip())
                    Reviews.append(rating_data[1].strip() if len(rating_data) > 1 else 0)
                else:
                    Rating.append('N/A')
                    Reviews.append('N/A')
            
            except Exception as e:
                print(f"Error processing item: {item.text.strip()}\nError: {e}")
    
    # Create a DataFrame with the scraped data
    all_data = {
        'Scrap_time': datetime.now(),
        'Product_Name': Products,
        'Price': Price,
        'Rating': Rating,
        'Reviews': Reviews
    }
    df = pd.DataFrame(all_data)
    return df

# Example Usage
url = 'https://www.meesho.com/accessories-men/pl/3tp'
scraped_df = scrap_meescho(url)
print(scraped_df)

                   Scrap_time                                 Product_Name  \
0  2024-11-18 22:48:37.013160                      +3 MoreElite Men Gloves   
1  2024-11-18 22:48:37.013160                     +4 MoreGorgeous Men Caps   
2  2024-11-18 22:48:37.013160  01h : 26m : 23sFashionable Trendy Men Socks   
3  2024-11-18 22:48:37.013160                       Fancy Trendy Men Socks   
4  2024-11-18 22:48:37.013160                     Fashionate Men Bracelets   
5  2024-11-18 22:48:37.013160               +3 MoreStyles Unique Men Socks   
6  2024-11-18 22:48:37.013160                 Fashionable Modern Men Socks   
7  2024-11-18 22:48:37.013160    +8 MoreFashionable Trendy Men Caps & Hats   
8  2024-11-18 22:48:37.013160          +8 MoreFancy Latest Men Caps & Hats   
9  2024-11-18 22:48:37.013160                      Casual Unique Men Socks   
10 2024-11-18 22:48:37.013160                      Casual Unique Men Socks   
11 2024-11-18 22:48:37.013160          +4 MoreFancy Modern Men C

In [14]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Replace with your local file
with open("mesooo_acce.html", 'r') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

all_class = [
    'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA eCJiSA',
    'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA jGAjKy',
    'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA bGpzcx',
    'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA ihwYlH'
]

Rating = []
Products = []
Price = []
Reviews = []
off_pri = []
dis = []
dilev_charg = []

for cls in all_class:
    for item in soup.find_all('div', {'class': cls}):
        scrap_data = item.text.strip().split('₹')

        if len(scrap_data) == 2:
            prod, other_details = item.text.strip().split('₹')
            rest_d, rat_rev = other_details.split(' Free Delivery')
            pric = rest_d

            Products.append(prod)
            Price.append(pric)
            off_pri.append(pric)
            dis.append(0)
            dilev_charg.append(0)
            if 'supplier' in rat_rev.lower():
                Rating.append(rat_rev[:3])
                Reviews.append(0)
            elif 'reviews' in rat_rev.lower():
                rat_re, _ = rat_rev.split(' ')
                Rating.append(rat_re[:3])
                Reviews.append(rat_re[3:])
            else:
                pass

        elif len(scrap_data) == 3:
            prod, offered_price, rest_details = scrap_data
            Products.append(prod)
            off_pri.append(offered_price)
            pprice_dic, rate = rest_details.split(' offFree Delivery')
            Price.append(pprice_dic[:3])
            dis.append(pprice_dic[3:])
            dilev_charg.append(0)
            ret_rev = scrap_data[2].split(" offFree Delivery")[1]
            rating = ret_rev.split(" ")
            Reviews.append(rating[0][3:])
            Rating.append(rating[0][:3])

        else:
            Products.append(scrap_data[0])
            off_pri.append(0)
            Price.append(scrap_data[1].split(" ")[0])
            dilev_charg.append(scrap_data[2])
            dis.append(0)
            rat_revies = scrap_data[3].split(" ")
            Rating.append(rat_revies[0][2:5])
            Reviews.append(rat_revies[0][5:])

# Create a DataFrame with the extracted data
all_data = {
    'Scrap_time': datetime.now(),
    'Product_Name': Products,
    'Offered_Price': off_pri,
    'Price': Price,
    'Discount': dis,
    'Rating': Rating,
    'Reviews': Reviews,
    'Delever_charge': dilev_charg
}

df = pd.DataFrame(all_data)

# Save to CSV
df.to_csv("full_mesho.csv", index=False)
print("Data Extracted successfully")

Data Extracted successfully


In [18]:
#working shell 
url='''https://www.meesho.com/home-decor/pl/3tl'''

response = r.get(url)
html_data = response.content
soup = BeautifulSoup(html_data,'lxml')

all_class = ['sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA eCJiSA','sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA jGAjKy',
             'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA bGpzcx','sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA ihwYlH']


Rating = []
Products = []
Price =[]
Reviews =[]


for cls in all_class:
    for item in soup.find_all('div',{'class':cls}):
        scrap_data = item.text.strip().split('₹')
        
      
        prod, other_details = item.text.strip().split('₹')
        rest_d,rat_rev=other_details.split('Delivery')
       # print(other_details.split('Delivery'))
        pric = rest_d[:3]   
        Products.append(prod)
        Price.append(pric)
        if 'supplier' in rat_rev.lower():
            Rating.append(rat_rev[:3])
          #  print(rat_rev[:3])
            Reviews.append(0)
        elif 'reviews' in rat_rev.lower():            
            Rating.append(rat_rev[:3])
            Reviews.append(rat_rev[3:-7])
            #print(rat_rev)
        else:
            Rating.append(0)
            Reviews.append(0)
       
         
                
                    
            
                  
"""print(len(Rating))
print(len(Products))
print(len(Price))
print(len(Reviews)) """

all_data = {'Scrap_time':datetime.now(),'Product_Name':Products,'Price':Price,'Rating':Rating,'Reviews':Reviews}
df = pd.DataFrame(all_data)
df
df.to_csv('mesho.csv',index = False)

In [32]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os

def soup_meesho(url):
    # Fetch the webpage
    response = r.get(url)
    html_data = response.content
    soup = BeautifulSoup(html_data, 'lxml')

    # Classes to target product sections
    all_class = [
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA eCJiSA',
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA jGAjKy',
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA bGpzcx',
        'sc-dkrFOg ProductList__GridCol-sc-8lnc8o-0 cokuZA ihwYlH'
    ]

    # Initialize data lists
    Rating = []
    Products = []
    Price = []
    Reviews = []
    off_pri = []
    dis = []
    dilev_charg = []

    for cls in all_class:
        for item in soup.find_all('div', {'class': cls}):
            scrap_data = item.text.strip().split('₹')

            if len(scrap_data) == 2:
                prod, other_details = scrap_data
                rest_d, rat_rev = other_details.split(' Free Delivery')
                pric = rest_d

                Products.append(prod)
                Price.append(pric)
                off_pri.append(pric)
                dis.append(0)
                dilev_charg.append(0)

                if 'supplier' in rat_rev.lower():
                    Rating.append(rat_rev[:3])
                    Reviews.append(0)
                elif 'reviews' in rat_rev.lower():
                    rat_re, _ = rat_rev.split(' ')
                    Rating.append(rat_re[:3])
                    Reviews.append(rat_re[3:])
                else:
                    Rating.append(0)
                    Reviews.append(0)

            elif len(scrap_data) == 3:
                prod, offered_price, rest_details = scrap_data
                Products.append(prod)
                off_pri.append(offered_price)
                pprice_dic, rate = rest_details.split(' offFree Delivery')
                Price.append(pprice_dic[:3])
                dis.append(pprice_dic[3:])
                dilev_charg.append(0)

                ret_rev = rest_details.split(" offFree Delivery")[1]
                rating = ret_rev.split(" ")
                Rating.append(rating[0][:3])
                Reviews.append(rating[0][3:])
            else:
                Products.append(scrap_data[0])
                off_pri.append(0)
                Price.append(scrap_data[1].split(" ")[0])
                dilev_charg.append(scrap_data[2])
                dis.append(0)
                rat_revies = scrap_data[3].split(" ")
                Rating.append(rat_revies[0][2:5])
                Reviews.append(rat_revies[0][5:])

    # Define filename from URL
    file_name = url.split("https://www.meesho.com/")[1].split("/")[0]

    # Prepare data dictionary
    all_data = {'Scrap_time': datetime.now(), 'Product_Name': Products, 'Offered_Price': off_pri, 'Price': Price,
                'Discount': dis, 'Rating': Rating, 'Reviews': Reviews, 'Delever_charge': dilev_charg}

    # Convert data to DataFrame
    table_new = pd.DataFrame(all_data)

    # Define folder and check if file exists
    folder_path = '/Users/nsherpa/iCloud Drive (Archive) - 1/Documents'
    folder_finders = os.listdir(folder_path)

    # Check if file exists and append data if it does
    if f'{file_name}.csv' in folder_finders:
        table_old = pd.read_csv(f"{folder_path}/{file_name}.csv")
        final_table = pd.concat([table_old, table_new])
        final_table.to_csv(f'{folder_path}/{file_name}.csv', index=False)
        print("Data Appended to the existing Table")
    else:
        # Save new data if file does not exist
        table_new.to_csv(f'{folder_path}/{file_name}.csv', index=False)
        print("Data Extracted Successfully")

    # Return the DataFrame
    return table_new

# Example usage
url = "https://www.meesho.com/lehengas/pl/3l6"
soup_meesho(url)

Data Extracted Successfully


,Scrap_time,Product_Name,Offered_Price,Price,Discount,Rating,Reviews,Delever_charge
0,2024-11-18 23:13:15.269721,Aakarsha Sensational Women Lehenga,505,554,9%,4.0,1108,0
1,2024-11-18 23:13:15.269721,+4 MoreBanita Graceful Women Lehenga,438,438,0,3.7,5763,0
2,2024-11-18 23:13:15.269721,+7 MoreMyra Fabulous Women Lehenga,552,575,4%,3.8,2452,0
3,2024-11-18 23:13:15.269721,Kashvi Alluring Women Lehenga,754,754,0,3.9,1597,0
4,2024-11-18 23:13:15.269721,+7 MoreJivika Sensational Women Lehenga,425,471,10%,4.1,1368,0
5,2024-11-18 23:13:15.269721,+3 MoreCharvi Sensational Women Lehenga,462,462,0,3.9,3379,0
6,2024-11-18 23:13:15.269721,+8 MoreAakarsha Sensational Women Lehenga,288,300,4%,3.7,1464,0
7,2024-11-18 23:13:15.269721,Alisha Refined Women Lehenga,490,490,0,3.9,1406,0
8,2024-11-18 23:13:15.269721,Fancy Designer Flared Embroidered Indo Western...,646,646,0,3.9,23336,0
9,2024-11-18 23:13:15.269721,+3 MoreFancy Designer Panelled Jacquard South ...,515,542,5%,4.2,4806,0
